# GRPO强化学习

In [1]:
from datasets import load_dataset
from trl import GRPOTrainer,GRPOConfig
import warnings
warnings.filterwarnings("ignore")

E:\web\LLM-tuning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\web\LLM-tuning\.venv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 1、加载数据集

In [2]:
dataset = load_dataset("meta-math/GSM8K_zh", split="train[:1000]").select_columns(column_names=['question_zh','answer_zh','answer_only'])

In [3]:
dataset

Dataset({
    features: ['question_zh', 'answer_zh', 'answer_only'],
    num_rows: 1000
})

## 2、数据预处理

In [4]:
def covert_data(example):
    return {'prompt':[{'role':'user','content':example['question_zh']}],'solution':example['answer_only']}

In [5]:
dataset=dataset.map(covert_data,remove_columns=dataset.column_names)

In [6]:
dataset

Dataset({
    features: ['prompt', 'solution'],
    num_rows: 1000
})

## 3、定义奖励函数

In [11]:
import re
def accuracy_reward(**kwargs):
    solution = str(kwargs["solution"]).strip()
    completions = kwargs["completions"]  # 注意这里

    rewards = []

    for comp in completions:
        output = comp.get("content", "")
        numbers = re.findall(r"\d+", output)
        if not numbers:
            rewards.append(0.0)
            continue

        pred = numbers[-1].strip()
        rewards.append(1.0 if pred == solution else 0.0)

    return rewards

## 3、配置GRPOConfig

In [12]:
args=GRPOConfig(
    output_dir='../../models/grpo_model',
    learning_rate=5e-6,
    per_device_train_batch_size=2,
    num_generations=2,
    num_train_epochs=1,
)

In [13]:
trainer = GRPOTrainer(
    model="Qwen/Qwen3-0.6B",
    reward_funcs=accuracy_reward,
    train_dataset=dataset,
)

Loading weights: 100%|██████████| 311/311 [00:00<00:00, 350.56it/s, Materializing param=model.norm.weight]                              
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
Some parameters are on the meta device because they were offloaded to the cpu.
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [14]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


AttributeError: 'list' object has no attribute 'get'